In [1]:
#---------------------------------------------------------------------------------------------LIBRARIES--------------------------------------------------------------------------------------------
import cv2                                                                                             #import OpenCV2 library for image processing and algorithms
import numpy as np                                                                                     #import numpy mathematical library
import operator                                                                                        #additional efficient pyhton fucntions 
import matplotlib.pyplot as plt                                                                        #import matplotlib library for plotting
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'G:/Program Files/Tesseract-OCR/tesseract.exe'                 #
from scipy import ndimage                                                                              #package contains various functions for multidimensional image processing                       
from webcolors import rgb_to_name, name_to_rgb                                                         #import the webcolors library which enables RGB to name and vice versa conversions
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))                                  #change width of Jupyer Notebook to use the whole window resolution available

In [2]:
#--------------------------------------------------------------------------------------------DEFINITIONS-------------------------------------------------------------------------------------------
def map_value(value, in_low, in_high, out_low, out_high):                                              #create Arduino map() function in python for usage throughout the code
    return out_low + (out_high - out_low) * ((value - in_low) / (in_high - in_low))                    #scale input lowest,input highest range to output lowest,output highest range then return

def digit_recognition(image):
    outcome = pytesseract.image_to_string(image,                                                       #use Pytesseract Engine to identify digits, convert output to integer
                         config='--psm 13 --oem 3 -c tessedit_char_whitelist=0123456789')              #load digit model and whitelist characters from 0 to 9 for identification
    return outcome                                                                                     #return recognized digit as string, or empty string if nothing has been recognized 

In [202]:
#----------------------------------------------------------------------------------------IMAGE PRE-PROCESSING---------------------------------------------------------------------------------------
colour_img = cv2.imread('./uno_images/bt.jpg')                                                         #load the image from the specified path
rotated_img = ndimage.rotate(colour_img, -90)                                                          #rotate image by 90 degrees, increase user ease of use
rotated_img_copy = rotated_img.copy()                                                                  #create a copy of the rotated_img so that they don't share the same memory address
bw_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)                                                 #convert to a black and white image


#--------------------------------------------------------------------------------------------BINARISATION-------------------------------------------------------------------------------------------
#img_sm = cv2.blur(img, (1, 1))                                                                        #not required for the images provided, no changes can be noted        
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_BINARY)                                 #accurate countours, seems to require more antialising
thr_value, th_img = cv2.threshold(bw_img,150, 400, cv2.THRESH_BINARY_INV)                              #accurate countours, smoother edges compared to regular binary
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_TRUNC)                                  #accurate canny that identifies depth, wrong contours
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_TOZERO)                                 #accurate canny with noise, issues with contours
#thr_value, img_th = cv2.threshold(bw_img,50, 100, cv2.THRESH_TOZERO_INV)                              #inaccurate countours 


#----------------------------------------------------------------------------------------MORPHOLOGY CORRECTION---------------------------------------------------------------------------------------
#very small changes with images provided, helps with countour accuracy
kernel = np.ones((3, 3), np.uint8)                                                                     #higher kernel = less accurate contours
#close_img = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)                                         #erosion + dilute method (internal spaces removal)
open_img = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, kernel)                                            #dilute + erosion method (noise removal)


#-----------------------------------------------------------------------------------EDGE DETENCTION & CONTOUR MAPPING---------------------------------------------------------------------------------
canny_img = cv2.Canny(open_img, 50, 100)                                                               #edge detection using the OpenCV Canny method
contours, _ = cv2.findContours(open_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)  
selected_contour = 1                                                                                   #0 = frame; 1 = uno card perimeter;
contours_img = cv2.drawContours(rotated_img, contours, selected_contour, (0,255,0), 2, cv2.LINE_AA)    #draw selected contour on top of the rotated colour image
#print(len(contours))                                                                                  #debugging: show how many contours have been found
#print(hierarchy)                                                                                      #debugging: show hierarchy list for all contours


#-------------------------------------------------------------------------------MANUAL CONTOUR CROPPING USING MAX FUNCTION-----------------------------------------------------------------------------
"""
max_coords = max((contours[selected_contour]).tolist())
contour_x = (max_coords[0])[0]
contour_y = (max_coords[0])[1]
print(contours[selected_contour])
print(max_coords)
print(contour_x)
print(contour_y)
"""


#---------------------------------------------------------------------------OPEN CV CONTOUR CROPPING USING CONTOURS FUNCTION--------------------------------------------------------------------------- 
x_cnt,y_cnt,w_cnt,h_cnt = cv2.boundingRect(contours[selected_contour])                                 #find origin, width and heigth of image based on selected_contour
contour_cropped_proc_img = rotated_img[y_cnt:y_cnt+h_cnt, x_cnt:x_cnt+w_cnt]                           #crop the image based on the coordinates found for processing
framed_img = cv2.copyMakeBorder(contour_cropped_proc_img,                                              #add a 30px wide black frame around the cropped image (helps with text placement) 
                                30,30,30,30,
                                cv2.BORDER_CONSTANT,
                                value=(0,0,0)) 
#print(x_cnt, y_cnt, w_cnt, h_cnt)                                                                     #print 4 values for debugging
 

#-------------------------------------------------------------------------------------------COLOUR ANALYSIS--------------------------------------------------------------------------------------------
contour_cropped_analysis_img = rotated_img_copy[y_cnt:y_cnt+h_cnt, x_cnt:x_cnt+w_cnt]                  #crop the rotated original image copy with the uno card contour for pattern/colour processing
#a patch of pixels is chosen to increase accuracy 
#as well as optimise the code by running through less pixels in the functions to follow
colour_patch_img = contour_cropped_analysis_img[40:80, 100:140]                                        #select a constant patch from the uno card image to perform the pixel analysis
bgr_pixels = colour_patch_img.tolist()                                                                 #transform patch array to list containing bgr tuples
#print(bgr_pixels)
b = [x[0][0] for x in bgr_pixels]                                                                      #extract blue values from bgr list
g = [x[0][1] for x in bgr_pixels]                                                                      #extract green values from bgr list
r = [x[0][2] for x in bgr_pixels]                                                                      #extract red values from bgr list
frequent_b = (max(set(b), key = b.count))                                                              #find the most frequent blue value in the image patch selected
frequent_g = (max(set(g), key = g.count))                                                              #find the most frequent green value in the image patch selected
frequent_r = (max(set(r), key = r.count))                                                              #find the most frequent red value in the image patch selected


#-----------------------------------------------------------------------------------------COLOUR IDENTIFICATION------------------------------------------------------------------------------------------    
rgb_dictionary = {"red": frequent_r, "green": frequent_g, "blue": frequent_b}                          #create dictionary containing the RGB colour space, and assign most frequent value from patch
sorted_rgb_dictionary = dict((y, x) for y, x in sorted(rgb_dictionary.items(),                         #sort dictionary based on value, not key and since the output is a tuple, transform to dictionary
                                                       key=operator.itemgetter(1)))                    #choose value for sorting process, (1) = value, (0) = key
highest_rgb_value = list(sorted_rgb_dictionary)[2]                                                     #extract highest value whether it is a blue, green or red pixel (key)
middle_rgb_value = list(sorted_rgb_dictionary)[1]                                                      #extract middle value whether it is a blue, green or red pixel (key)
lowest_rgb_value = list(sorted_rgb_dictionary)[0]                                                      #extract lowest value whether it is a blue, green or red pixel (key)

#the following part of the code changes the R,G,B values of the pixels to an extreme
#this helps identify the colour correctly with different levels of brightness in the image
colour_rgb = [0,0,0]                                                                                   #create list to store the new RGB values for colour identification
if sorted_rgb_dictionary.get(highest_rgb_value) >= 255/2:                                              #if the highest pixel value is higher than the mid-point, then:
    if highest_rgb_value == "red":                                                                     #if the highest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 255                                                                                #assign the first element in the colour_rgb = 255 
    elif highest_rgb_value == "green":                                                                 #if the highest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 255                                                                                #assign the second element in the colour_rgb = 255 
    elif highest_rgb_value == "blue":                                                                  #if the highest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 255                                                                                #assign the third element in the colour_rgb = 255 
elif sorted_rgb_dictionary.get(highest_rgb_value) <= 255/2:                                            #if the highest pixel value is lower than the mid-point, then:
    if highest_rgb_value == "red":                                                                     #if the highest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 128                                                                                #assign the first element in the colour_rgb = 128 
    elif highest_rgb_value == "green":                                                                 #if the highest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 128                                                                                #assign the second element in the colour_rgb = 128 
    elif highest_rgb_value == "blue":                                                                  #if the highest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 128                                                                                #assign the third element in the colour_rgb = 128 
if sorted_rgb_dictionary.get(middle_rgb_value) > 150:                                                  #if the middle pixel value is higher than 150, then:
    if middle_rgb_value == "red":                                                                      #if the middle_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 255                                                                                #assign the first element in the colour_rgb = 255 
    elif middle_rgb_value == "green":                                                                  #if the middle_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 255                                                                                #assign the second element in the colour_rgb = 255 
    elif middle_rgb_value == "blue":                                                                   #if the middle_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 255                                                                                #assign the third element in the colour_rgb = 255 
if sorted_rgb_dictionary.get(lowest_rgb_value) < 255/2:                                                #if the lowest pixel value is lower than the mid-point, then:
    if lowest_rgb_value == "red":                                                                      #if the lowest_rgb_value is a "red" pixel, then:
        colour_rgb[0] = 0                                                                                  #assign the first element in the colour_rgb = 0 
    elif lowest_rgb_value == "green":                                                                  #if the lowest_rgb_value is a "green" pixel, then:
        colour_rgb[1] = 0                                                                                  #assign the second element in the colour_rgb = 0 
    elif lowest_rgb_value == "blue":                                                                   #if the lowest_rgb_value is a "blue" pixel, then:
        colour_rgb[2] = 0                                                                                  #assign the third element in the colour_rgb = 0 
        
named_colour = rgb_to_name(colour_rgb)                                                                 #use webcolours library database to convert RGB to HEX and then to colour name in English 
colour_framed_img = cv2.copyMakeBorder(framed_img,7,7,7,7,cv2.BORDER_CONSTANT,                         #add a 7px wide frame at the edge of the frame with the same colour as the UNO card
                                       value=(colour_rgb[2],colour_rgb[1],colour_rgb[0]))              #elements 2,1,0 used because of BGR output, and RGB input
"""
#Debugging
test = name_to_rgb('cyan') 
print(test)
print(sorted_rgb_dictionary)
print(type(highest_rgb_value))
print(lowest_rgb_value, middle_rgb_value, highest_rgb_value)
print(colour, named_colour)
"""


#-------------------------------------------------------------------------------------RESOLUTION NORMALIZATION------------------------------------------------------------------------------------------
contour_cropped_copy = contour_cropped_analysis_img.copy()                                             #create a copy of the uno card contour cropped image
normalize_resolution = (300, 450)                                                                      #have a set resolution between all images (after cropping)
normalized_img = cv2.resize(contour_cropped_copy, normalize_resolution,                                #set image to be resized and the resolution
                                        interpolation = cv2.INTER_AREA)

#--------------------------------------------------------------------------------CENTER CROPPING FOR SHAPE DETECTION------------------------------------------------------------------------------------
ellipse = cv2.fitEllipse(contours[selected_contour])                                                   #fit an ellipse on the selected contour of the image (uno card contour)
(center, axes, orientation) = ellipse                                                                  #extract the main parameter
minoraxis_length = min(axes)
half_minor = int(minoraxis_length/2)
x_ellipse = center[0]
y_ellipse = center[1]
center_cropped_img = normalized_img[int(x_ellipse*0.95): int(y_ellipse), 
                                    int(x_ellipse-half_minor+30) : int(x_ellipse+half_minor-20)]

#print(majoraxis_length, minoraxis_length, x_ellipse, y_ellipse)


#------------------------------------------------------------------------------------CENTER CROPPED IMAGE SCALING---------------------------------------------------------------------------------------
scale = 250                                                                                            #percent of original size for scaling, 250%
dim = int(center_cropped_img.shape[1]*scale/100),int(center_cropped_img.shape[0]*scale/100)            #create variable storing the new resolution for scaling based on percentage
rescaled_img = cv2.resize(center_cropped_img, dim, interpolation = cv2.INTER_AREA)                     #rescale the center cropped image so that it is 2.5 times bigger (ease of use)


#------------------------------------------------------------------------------------SHAPE IMAGE POST-PROCESSING----------------------------------------------------------------------------------------
rescaled_img_bw = cv2.cvtColor(rescaled_img, cv2.COLOR_BGR2GRAY)                                       #convert rescaled center image to black and white channels for post-processing
thr_value2, th_digit_img = cv2.threshold(rescaled_img_bw,190, 250,cv2.THRESH_BINARY)              #accurate countours, smoother edges compared to regular binary
kernel_close = np.ones((3, 3), np.uint8)                                                               #higher kernel = less accurate contours
morph_digit_img = cv2.morphologyEx(th_digit_img, cv2.MORPH_CLOSE, kernel_close)                        #erosion + dilute method (internal spaces removal)
canny_shape = cv2.Canny(morph_digit_img, 50, 100)  
contours_shape, _shape = cv2.findContours(canny_shape, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
       
#print(len(contours_shape))
"""
perimeter2 = cv2.arcLength(contours_digits[2], True)   # contour length, also potentially useful
polygon_constant = 0.1    # try changing this value to get more or less corners
vertex_approx = len(cv2.approxPolyDP(contours_digits[3], polygon_constant*perimeter2, True)) 
"""

#------------------------------------------------------------------------------------FEATURE SPACE RECOGNITION------------------------------------------------------------------------------------------
area, perimeter = 0, 0                                                                                 #define variables called area and perimeter and assign them 0
polygon_constant = 0.005    # try changing this value to get more or less corners
if len(contours_shape) == 6:                                                                          #if the number of contours found in the center cropped image is higher than 5:
    for i in range(2,5):                                                                               #loop through contour 2 and 3 to get the total perimeter and area
        area += cv2.contourArea(contours_shape[i])
        perimeter += cv2.arcLength(contours_shape[i], True)
        shape_contours_img = cv2.drawContours(rescaled_img,                                            #every loop draw the i contour on top of the cropped image
                                              contours_shape, i, (0,255,0), 2, cv2.LINE_AA)           
    corners = len(cv2.approxPolyDP(contours_shape[-1], polygon_constant*perimeter, True))   
elif len(contours_shape) == 10:                                                                        #if the number of contours found in the center cropped image is higher than 6:
    for i in range(3,7):                                                                               #loop through contour 2,3 and 4 to get the total perimeter and area
        area += cv2.contourArea(contours_shape[i])
        perimeter += cv2.arcLength(contours_shape[i], True) 
        shape_contours_img = cv2.drawContours(rescaled_img,                                            #every loop draw the i contour on top of the cropped image
                                              contours_shape, i, (0,255,0), 2, cv2.LINE_AA)  
    corners = len(cv2.approxPolyDP(contours_shape[-1], polygon_constant*perimeter, True))   

      
#print(corners, area, perimeter)
"""
COMMENTS:
The reasoning behing the 2 statements is related to the amount of contours a SKIP card has, 
when compared to a REVERSE card or +2 card. 
Through print(area/perimeter) I found out that the ratios are:
    - SKIP CARD: 42<X<43
    - PLUS 2 CARD: is 39<X<40
    - REVERSE CARD: 24<X<25.
Note: The value is static, but the range simplifies the condition used later.
"""


#--------------------------------------------------------------------------------------DIGIT/SHAPE RECOGNITION-------------------------------------------------------------------------------------------
#haha = digit_recognition(morph_digit_img)
#print(haha)

#"""
digit_name = ""                                                                                        #create empty string storing the digit english name
digit = None

try:                                                                                                   #check for code errors when calling the definition
    digit = int(digit_recognition(center_framed_img))                                                    #in the conversion of the string to int (once digit has been recognized)
    #print(digit)
except ValueError:                                                                                     #returns "ValueError" when the digit has not been recognized = empty string = conversion error
    try:                                                                                               #check for errors again when running the following commands: 
        vertical_flipped_morph_digit_img = cv2.flip(center_framed_img,0)                                 #flip the digit detection(after morphology) image vertically 
        colour_framed_img = cv2.flip(colour_framed_img,0)                                              #also vertically flip the output final image for user ease of use
        digit = int(digit_recognition(vertical_flipped_morph_digit_img))                               #do the conversion again, if no error                                                                                            
    except ValueError:
        horizontal_flipped_morph_digit_img = cv2.flip(vertical_flipped_morph_digit_img,1)              #flip the digit detection image, with a vertical flip, horizontally for full correct alignment
        colour_framed_img = cv2.flip(colour_framed_img,1)                                              #also vertically flip the output final image for user ease of use
        try:
            digit = int(digit_recognition(horizontal_flipped_morph_digit_img))                         #do the conversion again, if no error   
            print(digit)
        except ValueError:                                                                             #if there is another ValueError, it means we have a card without a digit (reverse, +2, skip)  
            if (area/perimeter) > 26.0 and (area/perimeter) < 27.0:                                    #if the ratio between area and perimeter is between 24.0 and 25.0:
                digit_name = "reverse"                                                                     #assign the digit_name string the word "reverse"
            elif (area/perimeter) > 42.0 and (area/perimeter) < 43.0:                                  #if the ratio between area and perimeter is between 39.0 and 40.0:
                digit_name = "plus 2"                                                                      #assign the digit_name string the word "plus 2"
            elif (area/perimeter) > 40.0 and (area/perimeter) < 41.0:                                  #if the ratio between area and perimeter is between 42.0 and 43.0:
                digit_name = "skip"                                                                        #assign the digit_name string the word "skip"         
        else: 
            digit_name = digit
    else: 
        digit_name = digit
else: 
    digit_name = digit
print(digit_name)
#Method 1: If and Elif statements to get the correct name for the digit recognized
if (digit) == 0:                                                                                       #if the digit identified is 0, then: 
    digit_name = "zero"                                                                                #assign to empty string the text "zero"
elif (digit) == 1:                                                                                     #if the digit identified is 1, then: 
    digit_name = "one"                                                                                 #assign to empty string the text "one"
elif (digit) == 2:                                                                                     #if the digit identified is 2, then: 
    digit_name = "two"                                                                                     #assign to empty string the text "two"
elif (digit) == 3:                                                                                     #if the digit identified is 3, then: 
    digit_name = "three"                                                                                   #assign to empty string the text "three"
elif (digit) == 4:                                                                                     #if the digit identified is 4, then: 
    digit_name = "four"                                                                                    #assign to empty string the text "four"
elif (digit) == 5:                                                                                     #if the digit identified is 5, then: 
    digit_name = "five"                                                                                    #assign to empty string the text "five"
elif (digit) == 6:                                                                                     #if the digit identified is 6, then:  
    digit_name = "six"                                                                                     #assign to empty string the text "six"
elif (digit) == 7:                                                                                     #if the digit identified is 7, then: 
    digit_name = "seven"                                                                                   #assign to empty string the text "seven"
#lines 213 to 216 help with the skip and 8 shape differentiation in case of bad recognition
elif (digit) == 8 and (area/perimeter) > 43.0 and (area/perimeter) < 45.0:                             #if the digit identified is 8, with the correct ratio for an 8, then: 
    digit_name = "eight"                                                                                   #assign to empty string the text "eight"
elif (digit) == 8 and (area/perimeter) > 40.0 and (area/perimeter) < 41.0:                             #if the digit identified is 8, with the correct ratio for a "skip" then: 
    digit_name = "skip"                                                                                   #assign to empty string the text "skip"
elif (digit) == 9:                                                                                     #if the digit identified is 8, then: 
    digit_name = "nine"                                                                                    #assign to empty string the text "nine"

#"""
"""
#Method 2: A more elegant solution of Method 1 with a dictionary looped, if else condensed and parsing condition 
colour_name_dictionary = {"zero" : 0, "one" : 1, "two" : 2, "three" : 3, "four" : 4, "five" : 5, "six" : 6, "seven" : 7, "eight" : 8, "nine" : 9}
if digit is not None:
    keys = [key for key, value in colour_name_dictionary.items() if value == digit]                               
    digit_name = (str(keys))[2:-2] if 0 <= digit <= 9 and not 42.0 < (area/perimeter) < 43.0 else "skip"   

#print(keys)   
#print(type(digit), digit)                                                                             #print the type of the variable and its contents
#print(digit_name)                                                                                     #print string to check correct output
"""
#------------------------------------------------------------------------------------IDENTIFICATION TEXT PLACEMENT--------------------------------------------------------------------------------------                                                                      
disp_x = +240                                                                                          #perfect value = +240
disp_y = -165                                                                                          #perfect value = -165
text_displayed = (named_colour + " " + digit_name + " uno card").upper()
identified_img = cv2.putText(colour_framed_img,                                                        #create text on top of image
                             text_displayed,                                                           #set text to text_displayed string
                             (y_cnt+disp_y, x_cnt+disp_x),                                             #set coords to contour bottom right corner with displacement
                             cv2.FONT_HERSHEY_SIMPLEX,                                                 #set OpenCV font
                             0.4, (0,255,0), 1, cv2.LINE_AA)                                           #font size, colour, thickness, antialiasing on the text for smoother edges


#------------------------------------------------------------------------------------POST-PROCESSED IMAGE SCALLING-------------------------------------------------------------------------------------                                                                                             
scaling = int(identified_img.shape[1]*1.5),int(identified_img.shape[0]*1.5)                            #scale the output image by 1.5x
final_img = cv2.resize(identified_img, scaling, interpolation = cv2.INTER_AREA)                        #scale the identified_img, with scaling resolution


#-------------------------------------------------------------------------------------------IMAGE DISPLAY---------------------------------------------------------------------------------------------- 
cv2.imshow('original', colour_img)                                                                     #show the original image to the user
cv2.imshow('cropped', contour_cropped_analysis_img)                                                    #show the contour cropped image to the user
cv2.imshow('final_img', final_img)                                                                     #show the final image to the user


#Debugging digit extraction
cv2.imshow('rez_normalized',normalized_img)
cv2.imshow('morphology',morph_digit_img)
cv2.imshow('center_cropped',center_cropped_img)
cv2.imshow('center framed',center_framed_img)
cv2.imshow('canny shape', canny_shape)
cv2.imshow('flipped2', horizontal_flipped_morph_digit_img)
cv2.imshow('binarisation shape', th_digit_img)
cv2.imshow('contours shape', shape_contours_img)


key = cv2.waitKey(0)                                                                                   #wait for any key press
cv2.destroyAllWindows()                                                                                #close all windows displaying images